<a href="https://colab.research.google.com/github/Cld338/15MinNeighbourHoods/blob/main/15%EB%B6%84_%EB%8F%84%EC%8B%9C_%EC%A1%B0%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
import pandas as pd
import os
path = "/content/gdrive/MyDrive/SW알고리즘/"
LINK_F = pd.read_csv(f"{path}/MOCT_LINK_filtered.csv", encoding="cp949")
NODE_F = pd.read_csv(f"{path}/MOCT_NODE_filtered.csv", encoding="cp949")
SPD_F = pd.read_csv(f"{path}/speed.csv", encoding="cp949")


SPD_DICT = {}
for i in range(SPD_F.shape[0]):
    row = SPD_F.loc[i]
    if row.loc["시작노드"] not in SPD_DICT.keys():
        SPD_DICT[row.loc["시작노드"]] = {}
    SPD_DICT[row.loc["시작노드"]][row.loc["끝노드"]] = {"TIME":row.loc["주행시간"]}


GPS_DICT = {}   
for i in range(NODE_F.shape[0]):
    row = NODE_F.loc[i]
    if row.loc["NODE_ID"] not in GPS_DICT.keys():
        GPS_DICT[row.loc["NODE_ID"]] = []
    GPS_DICT[row.loc["NODE_ID"]] = tuple(list(map(float, (row.loc["geometry"].replace("POINT (", "").replace(")", "").split()))))


ROAD_DICT = {}
for i in range(LINK_F.shape[0]):
    row = LINK_F.loc[i]
    if row.loc["F_NODE"] not in ROAD_DICT.keys():
        ROAD_DICT[row.loc["F_NODE"]] = []
    try:
        ROAD_DICT[row.loc["F_NODE"]] += [{
            "T_NODE" : row.loc["T_NODE"], 
            "TIME" : SPD_DICT[row.loc["F_NODE"]][row.loc["T_NODE"]]["TIME"],
            "GPS" : (GPS_DICT[row.loc["F_NODE"]], GPS_DICT[row.loc["T_NODE"]])
        }]
    except KeyError:
        pass
print(ROAD_DICT)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108
7109
7110
7111
7112
7113
7114
7115
7116
7117
7118
7119
7120
7121
7122
7123
7124
7125
7126
7127
7128
7129
7130
7131
7132
7133
7134
7135
7136
7137
7138
7139
7140
7141
7142
7143
7144
7145
7146
7147
7148
7149
7150
7151
7152
7153
7154
7155
7156
7157
7158
7159
7160
7161
7162
7163
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7176
7177
7178
7179
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7193
7194
7195
7196
7197
7198
7199
7200
7201
7202
7203
7204
7205
7206
7207
7208
7209
7210
7211
7212
7213
7214
7215
7216
7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
7243
7244
7245
7246
7247
7248
7249
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266
7267
7268

In [4]:
useable_ls = []
# GPS가 등록된 도로만 걸러내기
for i in ROAD_DICT.keys():
    if ROAD_DICT[i]:
        if ROAD_DICT[i][0]["GPS"]:
            useable_ls.append(i)


In [5]:
import folium
import sys
sys.setrecursionlimit(25000)

In [6]:
#현재 노드에서 남은 시간동안 도달할 수 있는 노드를 ls에 저장한다
def check_reachable_nodes(start_node, current_time=0):
    global k
    k+=1
    print(k)
    for link in ROAD_DICT[start_node]:
        if (start_node,link["T_NODE"]) in visited_link:
            continue
        elif current_time + link["TIME"] < 900:
            visited_link.append((start_node, link["T_NODE"]))
            NODE_TIME[link["T_NODE"]] = current_time + link["TIME"]
            ls.append([link["T_NODE"], link["TIME"], link["GPS"][1]])
            check_reachable_nodes(link["T_NODE"], current_time + link["TIME"])

In [7]:
# NODE = 4060186801
# NODE_TIME = {} #특정 노드에 도달하는데 걸리는 시간 "노드 ID" : "소요 시간"
# print(ROAD_DICT[NODE])
# start_node = ROAD_DICT[NODE][0]["GPS"][0]
# ls=[]
# k = 0
# visited_link = []

# check_reachable_nodes(NODE,0)
# print(len(ls))
# print(ls)
# mpp = folium.Map(location=[33.4, 126.5], tiles='openstreetmap', zoom_start=10)
# for node in ls:
#     folium.Marker(location=[node[2][1], node[2][0]], icon=folium.Icon(color='blue',icon='star')).add_to(mpp)
# folium.Marker(location=[start_node[1], start_node[0]], icon=folium.Icon(color='red',icon='star')).add_to(mpp)

In [8]:

#학교 노드 등록
SCHOOL_F = pd.read_csv(f"{path}/school_data_filtered.csv", encoding="cp949")
elementry_school_ls = []
middle_school_ls = []
high_school_ls = []
for i in range(SCHOOL_F.shape[0]):
  row = SCHOOL_F.loc[i]

  if row.loc["학교급구분"] == "초등학교":
      elementry_school_ls.append((row.loc["학교명"], (row.loc["위도"], row.loc["경도"])))
  elif row.loc["학교급구분"] == "중학교":
      middle_school_ls.append((row.loc["학교명"], (row.loc["위도"], row.loc["경도"])))
  elif row.loc["학교급구분"] == "고등학교":
      high_school_ls.append((row.loc["학교명"], (row.loc["위도"], row.loc["경도"])))
#(학교명, [위도, 경도])



#병원 노드 등록
HOSPITAL_F = pd.read_csv(f"{path}/hospital_data_filtered.csv", encoding="cp949")
hospital_ls = []
for i in range(HOSPITAL_F.shape[0]):
  row = HOSPITAL_F.loc[i]
  loc = list(map(float, row.loc["geometry"].replace("POINT (","").replace(")","").split()))
  hospital_ls.append((None, [loc[1], loc[0]]))
#(None, [위도 경도])


In [10]:
def check_reachable_nodes(start_node, current_time, time_limit):
    for link in ROAD_DICT[start_node]:
        if (start_node,link["T_NODE"]) in visited_link:
            continue
        elif current_time < time_limit: #+ link["TIME"] 
        #else:
            visited_link.append((start_node, link["T_NODE"]))
            visited_nodes_GPS.append(link["GPS"])
            #print(visited_nodes_GPS)
            NODE_TIME[link["T_NODE"]] = current_time + link["TIME"]
            ls.append([link["T_NODE"], link["TIME"], link["GPS"]])
            check_reachable_nodes(link["T_NODE"], current_time + link["TIME"], time_limit)

# 15분 내에 도달 가능한 노드를 찾고, 이를 마커로 표시하는 함수
def draw_all(NODE, time_limit):
  visited_nodes_GPS = []
  try:
    start_node = ROAD_DICT[NODE][0]["GPS"][0]
  except IndexError:
    return
  check_reachable_nodes(NODE,0,time_limit)

  #draw_alphashape(visited_nodes_GPS)
  for link in ls:
      if link not in drawn_links:
          drawn_links.append(link)
          folium.PolyLine(
              locations=[[link[2][0][1], link[2][0][0]], [link[2][1][1], link[2][1][0]]], 
              tooltip="Polyline",
              weight=4,
              popup=(f"{link[0]}, {link[2]}")
          ).add_to(mpp)
      #folium.Circle(location=[node[2][1][1], node[2][1][0]], radius=200, color="red")
  
  
"""
elementry_school_ls : 초등학교
hospital_ls : 병원
"""

'\nelementry_school_ls : 초등학교\nhospital_ls : 병원\n'

In [12]:
import sys
import time
sys.setrecursionlimit(100000)
mpp = folium.Map(location=[33.4, 126.5], tiles='openstreetmap', zoom_start=10)
#4060186801 - 반복
NODE_TIME = {} #특정 노드에 도달하는데 걸리는 시간 "노드 ID" : "소요 시간"

ls=[]
#DFS
k = 0
visited_link = []
drawn_links = []
l = len(middle_school_ls)
for i in range(l):
  school = middle_school_ls[i]
  print(f"{i+1}/{l}, {school[0]}")
  folium.Marker(location=school[1], icon=folium.Icon(color='blue',icon='star'), popup=school[0]).add_to(mpp)
  nodeID, nodeGPS = nearest_node(school[1])
  draw_all(nodeID,900)
mpp

1/92, 대정중학교
2/92, 성산중학교
3/92, 무릉중학교
4/92, 안덕중학교
5/92, 한림중학교
6/92, 신창중학교
7/92, 귀일중학교
8/92, 김녕중학교
9/92, 저청중학교
10/92, 제주대학교사범대학부설중학교
11/92, 제주중앙여자중학교
12/92, 위미중학교
13/92, 신성여자중학교
14/92, 제주여자중학교
15/92, 한림여자중학교
16/92, 노형중학교
17/92, 남원중학교
18/92, 탐라중학교
19/92, 서귀중앙여자중학교
20/92, 오현중학교
21/92, 오름중학교
22/92, 세화중학교
23/92, 서귀포여자중학교
24/92, 신엄중학교
25/92, 서귀포대신중학교
26/92, 제주중앙중학교
27/92, 우도중학교
28/92, 중문중학교
29/92, 남주중학교
30/92, 아라중학교
31/92, 세인트존스베리 아카데미 제주
32/92, 함덕중학교
33/92, 한라중학교
34/92, 조천중학교
35/92, 제주동중학교
36/92, 서귀포중학교
37/92, 애월중학교
38/92, 제주서중학교
39/92, 고산중학교
40/92, 제주중학교
41/92, 제주제일중학교
42/92, 한국국제학교
43/92, 효돈중학교
44/92, 제주동여자중학교
45/92, 표선중학교
46/92, 신산중학교
47/92, 대정중학교
48/92, 성산중학교
49/92, 무릉중학교
50/92, 제주중앙여자중학교
51/92, 제주대학교사범대학부설중학교
52/92, 김녕중학교
53/92, 한림중학교
54/92, 안덕중학교
55/92, 저청중학교
56/92, 신창중학교
57/92, 귀일중학교
58/92, 탐라중학교
59/92, 한림여자중학교
60/92, 오현중학교
61/92, 서귀중앙여자중학교
62/92, 제주여자중학교
63/92, 오름중학교
64/92, 노형중학교
65/92, 위미중학교
66/92, 남원중학교
67/92, 신성여자중학교
68/92, 남주중학교
69/92, 우도중학교
70/92, 중문중학교
71/92, 제주중앙중학교
72/92, 세화중학

In [ ]:
mpp

In [13]:
import sys
import time
sys.setrecursionlimit(100000)
mpp = folium.Map(location=[33.4, 126.5], tiles='openstreetmap', zoom_start=10)
#4060186801 - 반복
NODE_TIME = {} #특정 노드에 도달하는데 걸리는 시간 "노드 ID" : "소요 시간"

ls=[]
#DFS
k = 0
visited_link = []

drawn_links = []

l = len(high_school_ls)

for i in range(l):
  hospital = high_school_ls[i]
  print(f"{i+1}/{l}, {hospital[0]}")
  folium.Marker(location=hospital[1], icon=folium.Icon(color='blue',icon='star'), popup=f"<pre>{hospital[0]}</pre>").add_to(mpp)
  nodeID, nodeGPS = nearest_node(hospital[1])
  draw_all(nodeID,1500)

1/68, 제주과학고등학교
2/68, 영주고등학교
3/68, 중문고등학교
4/68, 한림고등학교
5/68, 삼성여자고등학교
6/68, 제주여자고등학교
7/68, 제주고등학교
8/68, 제주중앙여자고등학교
9/68, 남녕고등학교
10/68, 서귀포고등학교
11/68, 세화고등학교
12/68, 오현고등학교
13/68, 성산고등학교
14/68, NLCS JeJu
15/68, 한국뷰티고등학교
16/68, 표선고등학교
17/68, 애월고등학교
18/68, 함덕고등학교
19/68, 제주제일고등학교
20/68, 한림공업고등학교
21/68, 제주중앙고등학교
22/68, 제주외국어고등학교
23/68, 대정고등학교
24/68, 서귀포여자고등학교
25/68, 대정여자고등학교
26/68, 제주여자상업고등학교
27/68, 대기고등학교
28/68, KIS High School
29/68, 신성여자고등학교
30/68, 제주대학교사범대학부설고등학교
31/68, 서귀포산업과학고등학교
32/68, Branksome Hall Asia
33/68, 남주고등학교
34/68, 세인트존스베리 아카데미 제주
35/68, 제주과학고등학교
36/68, 삼성여자고등학교
37/68, 한림고등학교
38/68, 중문고등학교
39/68, 영주고등학교
40/68, 세화고등학교
41/68, 오현고등학교
42/68, 성산고등학교
43/68, 서귀포고등학교
44/68, 제주여자고등학교
45/68, 제주고등학교
46/68, 남녕고등학교
47/68, 제주중앙여자고등학교
48/68, 표선고등학교
49/68, 제주제일고등학교
50/68, 함덕고등학교
51/68, 한국뷰티고등학교
52/68, 애월고등학교
53/68, NLCS JeJu
54/68, 제주여자상업고등학교
55/68, KIS High School
56/68, 서귀포여자고등학교
57/68, 대기고등학교
58/68, 대정여자고등학교
59/68, 대정고등학교
60/68, 제주외국어고등학교
61/68, 제주중앙고등학교
62/68, 한림공업고등학교
63/68, 남주고등학교
64/

In [ ]:
mpp

In [15]:
# mpp.save("highschool_map.html")